# Add injury to insult
*A case study of murdering someone with the right choice of words.*

In [ ]:
# Install in jupyterlite
%pip install -q pathfinder2e-stats

import numpy as np
import xarray

import pathfinder2e_stats as pf2

## Attacker
Nyah, level 5 witch (The Resentment)
**Skills** Diplomacy +14 (Bon Mot)

**Occult Spells** DC 21; **3rd** Paralyze, Biting Words; **2nd** Blistering Invective; **1st** Sure Strike; **Cantrips (3rd)** Evil Eye, Guidance

## Attack routine
1. Bon Mot ➡ Blistering Invective
2. Paralyze ➡ Evil Eye
3. Evil Eye ➡ Biting Words
4. Evil Eye ➡ Sure Strike ➡ Biting Words attack
5. Evil Eye ➡ Guidance ➡ Biting Words attack

## Assumptions
- The target attempts to clear neither Bon Mot nor Sickened
- No movement is needed; the target remains within 30ft at all times
- Spellcasting is not disrupted or obstructed in any way
- Ignoring damage dealt to other creatures by casting heightened blistering invective

In [ ]:
level = 5

diplomacy = (
    (
        pf2.tables.PC.level
        + pf2.tables.PC.skill_proficiency.others.sel(priority=1)
        + pf2.tables.PC.ability_bonus.boosts.sel(initial=3)
        + pf2.tables.PC.skill_item_bonus.diplomacy
    )
    .sel(level=level)
    .item()
)

spell_DC = pf2.tables.SIMPLE_PC.spell_DC.witch.sum("component").sel(level=level).item()

print(f"{diplomacy=} {spell_DC=}")

In [ ]:
# You can change any of these to upcast or downcast them;
# damage and incapacitation trait are adjusted automatically
blistering_invective_rank = 2
paralyze_rank = 3
biting_words_rank = 3

## Targets

In [ ]:
targets = xarray.Dataset(
    {
        "target": [
            "The Stag Lord",
            "Ettin",
            "Vampire Count",
            "Hill Giant",
            "Dweomercat",
            "Sphinx",
        ],
        "level": ("target", [6, 6, 6, 7, 7, 8]),
        "HP": ("target", [110, 110, 65, 140, 100, 135]),
        "AC": ("target", [23, 21, 24, 24, 25, 27]),
        "Will": ("target", [9, 12, 17, 13, 17, 19]),
        "bonus_save_vs_magic": ("target", [0, 0, 0, 0, 1, 0]),
        "sickened": ("target", [1, 0, 0, 0, 0, 0]),
    }
)
targets["rank"] = pf2.level2rank(targets.level)
targets.to_pandas()

As this is a what-if analysis, roll a single d20 and compare it against different targets.
We don't want to repeat `dependent_dims=["target"]` for every call of check() and damage(); so we're going to set it as thread-wide configuration. As we're going to analyse multiple rounds and we're going to use a ``round`` independent dimension later, let's take the opportunity to configure it now too.

In [ ]:
pf2.set_config(
    check_independent_dims=["round"],
    check_dependent_dims=["target"],
    damage_independent_dims=["round"],
    damage_dependent_dims=["target"],
)

## Attack routine
### Round 1: Bon Mot ➡ Blistering Invective

In [ ]:
bon_mot = pf2.check(
    diplomacy,
    DC=targets.Will + 10 - targets.sickened,
)
bon_mot["Will_penalty"] = pf2.map_outcome(
    bon_mot.outcome,
    {pf2.DoS.success: 2, pf2.DoS.critical_success: 3},
)

sickened = [targets.sickened]
will = [
    pf2.sum_bonuses(
        ("untyped", targets.Will),
        ("status", targets.bonus_save_vs_magic),
        ("status", -targets.sickened),
        ("status", -bon_mot.Will_penalty),
    )
]

In [ ]:
blistering_invective_spec = pf2.armory.spells.blistering_invective(
    blistering_invective_rank
)
blistering_invective_spec

In [ ]:
blistering_invective = pf2.damage(
    pf2.check(will[0], DC=spell_DC),
    blistering_invective_spec,
    persistent_damage_rounds=5,
).rename({"persistent_round": "round"})

blistering_invective_damage = (
    blistering_invective["persistent_damage"]
    .where(blistering_invective["apply_persistent_damage"], 0)
    .sum("damage_type")
)

Probability of being on fire, by target by round

In [ ]:
(blistering_invective_damage > 0).mean("roll").round(2).to_pandas()

Mean damage of Blistering Invective every round (assuming no actions are spent putting the fire out)

In [ ]:
blistering_invective_damage.mean("roll").round(2).to_pandas()

In [ ]:
frightened = pf2.map_outcome(
    blistering_invective["outcome"],
    {pf2.DoS.failure: 1, pf2.DoS.critical_failure: 2},
)
# The frightened condition decays with every round that passes
frightened = np.maximum(0, frightened - blistering_invective["round"])

In [ ]:
roll_with_high_frightened = np.unique(
    frightened,
    return_index=True,
    axis=frightened.dims.index("roll"),
)[1][-2]
frightened.isel(roll=roll_with_high_frightened).to_pandas()

### Round 2: Paralyze ➡ Evil Eye

In [ ]:
will.append(
    pf2.sum_bonuses(
        ("untyped", targets.Will),
        ("status", targets.bonus_save_vs_magic),
        ("status", -sickened[-1]),
        ("status", -bon_mot.Will_penalty),
        ("status", -frightened.isel(round=1, drop=True)),
    )
)
paralyze = pf2.check(
    bonus=will[-1],
    DC=spell_DC,
    incapacitation=targets["rank"] > paralyze_rank,
)

# In case of failure, we use Evil Eye to extend the paralysis for the whole combat
paralyze["need_evil_eye"] = paralyze.outcome <= pf2.DoS.failure
paralyze["off_guard"] = paralyze.outcome <= pf2.DoS.failure

Probability of the target being paralyzed, as well as of needing to cast Evil Eye every round in order to maintain the condition for the whole fight.

**FIXME:** a critical failure on the initial saving throw followed by a critical success on any of the consecutive rounds will cause the target to snap out of paralysis early. This is not modelled here yet.

In [ ]:
paralyze["need_evil_eye"].mean("roll").round(2).to_pandas().to_frame("% paralyzed")

In [ ]:
def evil_eye(will_bonus, spell_DC):
    c = pf2.check(will_bonus, DC=spell_DC).outcome
    return pf2.map_outcome(c, {pf2.DoS.critical_failure: 2, pf2.DoS.failure: 1})


sickened.append(np.maximum(sickened[-1], evil_eye(will[-1], spell_DC)))

### Round 3: Evil Eye ➡ Biting Words
If the target scored a simple failure vs. Paralyze in round 2, extend its duration with Evil Eye.
Then, cast Biting Words.
### Round 4: Evil Eye ➡ Sure Strike ➡ Biting Words attack
### Round 5: Evil Eye ➡ Guidance ➡ Biting Words attack

In [ ]:
for _round in range(2, 5):
    will.append(
        pf2.sum_bonuses(
            ("untyped", targets.Will),
            ("status", targets.bonus_save_vs_magic),
            ("status", -sickened[-1]),
            ("status", -bon_mot.Will_penalty),
        )
    )
    sickened.append(np.maximum(sickened[-1], evil_eye(will[-1], spell_DC)))

assert len(will) == 5
assert len(sickened) == 5

will = xarray.concat(will, dim="round")
sickened = xarray.concat(sickened, dim="round")
sure_strike = xarray.DataArray([False, False, False, True, False], dims=["round"])
guidance = xarray.DataArray([False, False, False, False, True], dims=["round"])

Mean Will saves debuff by target and round

In [ ]:
(will - targets.Will).mean("roll").round(2).T.to_pandas()

In [ ]:
off_guard = xarray.concat(
    [
        xarray.DataArray(False),
        paralyze.off_guard.expand_dims(round=4),
    ],
    dim="round",
)
AC = pf2.sum_bonuses(
    ("untyped", targets.AC),
    ("status", -frightened),
    ("status", -sickened),
    ("circumstance", off_guard.astype(int) * -2),
)

Mean Armor Class debuff by target and round

In [ ]:
(AC - targets.AC).mean("roll").round(2).sel(drop=True).to_pandas()

In round 3, we use Sure Strike only if we don't need to extend the duration of paralyze.
In round 4 and 5, we always use Sure Strike.

In [ ]:
biting_words_check = pf2.check(
    spell_DC - 10 + guidance,
    DC=AC,
    fortune=sure_strike,
)
biting_words_check["outcome"] = biting_words_check["outcome"].where(
    AC["round"] >= 2, pf2.DoS.no_roll
)

biting_words_damage = pf2.damage(
    biting_words_check,
    pf2.armory.spells.biting_words(biting_words_rank),
).total_damage

Mean damage of Biting Words by target and round

In [ ]:
(biting_words_damage.mean("roll").round(2).to_pandas())

## Put it all together

In [ ]:
final = xarray.Dataset(
    {
        "AC": AC,
        "Will": will,
        "off_guard": paralyze.off_guard,
        "need_evil_eye": paralyze.need_evil_eye,
        "blistering_invective": blistering_invective_damage,
        "biting_words": biting_words_damage,
        "total_damage": blistering_invective_damage + biting_words_damage,
    }
).transpose("target", "roll", "round")
final["harmed"] = final.total_damage.sum("round") > 0
final["bloodied"] = final.total_damage.sum("round") > targets.HP // 2
final["killed"] = final.total_damage.sum("round") >= targets.HP
final

## Let's analyse our results!
### Mean cumulative damage by the end of the attack routine

In [ ]:
(
    final[["blistering_invective", "biting_words", "total_damage"]]
    .mean("roll")
    .sum("round")
    .round(2)
    .to_pandas()
)

### Various probabilities

- Probability of dealing any HP damage at all
- Probability of dealing more than 50% HP damage
- Probability of solo killing the target
- Probability of paralyzing the target in round 2
- Probability of needing to spam evil eye every round to keep the target paralyzed

In [ ]:
(
    final[["harmed", "bloodied", "killed", "off_guard", "need_evil_eye"]]
    .mean("roll")
    .round(2)
    .to_pandas()
)

### Damage distribution, normalized by target's Hit Points total

In [ ]:
_ = (
    (final["total_damage"].sum("round") / targets.HP)
    .to_pandas()
    .T.describe()
    .T.plot(
        kind="barh",
        y="mean",
        xerr="std",
        legend=False,
        title="Damage as % of total HP after round 5; mean+stddev",
    )
)